In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredText
import plotly.express as px
from sqlalchemy import create_engine
from statsmodels.tsa.seasonal import STL
from statsforecast import StatsForecast
from statsforecast.models import MSTL, AutoARIMA
from utilsforecast.plotting import plot_series
from datetime import date, timedelta
import calendar
from coreforecast.scalers import boxcox_lambda, boxcox, inv_boxcox
from scipy.stats import entropy
import pickle

c:\Users\joana\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys
import os

# Adiciona a pasta-pai ao sys.path (para importar meteorology)
sys.path.append(os.path.abspath(".."))
from construcao_modelos.Mstl_contadores_todos import GetMSTL
df_anomalias_total = GetMSTL()

In [3]:
df_corrigido_final = pd.read_csv(r"C:\Users\joana\Desktop\Projeto\construcao_modelos\df_corrigido_final.csv")

In [16]:
df_corrigido_final

,detected,direction_2,count_2,direction_3,count_3,direction_4,count_4,direction_5,count_5,direction_6,...,count_32,direction_33,count_33,direction_34,count_34,direction_35,count_35,holiday,weekend,exog
0,2021-08-01,219,327,155,323,138,424,238,341,539,...,500,401,769,354,691,526,1149,0,1,1
1,2021-08-02,144,252,194,381,117,372,249,364,735,...,486,355,590,436,777,523,1163,0,0,0
2,2021-08-03,151,257,187,397,137,411,321,431,732,...,568,441,699,376,724,584,1194,0,0,0
3,2021-08-04,141,266,157,337,158,423,306,404,706,...,563,379,626,430,799,616,1236,0,0,0
4,2021-08-05,159,259,182,333,121,370,257,370,740,...,584,367,602,424,846,636,1245,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,2024-02-25,298,583,45,146,113,188,123,339,192,...,239,187,583,273,776,198,548,0,1,1
939,2024-02-26,226,494,290,606,193,316,281,732,357,...,206,155,374,381,1214,221,608,0,0,0
940,2024-02-27,235,539,347,743,240,377,428,1053,433,...,285,344,770,468,1368,271,780,0,0,0
941,2024-02-28,249,573,254,572,261,437,379,972,456,...,317,352,829,460,1399,267,744,0,0,0


In [4]:
import dash
import jupyter_dash
print("Dash version:", dash.__version__)
print("JupyterDash version:", jupyter_dash.__version__)


Dash version: 3.0.4
JupyterDash version: 0.4.2


In [20]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import pandas as pd

# Garantir datetime
df_corrigido_final['detected'] = pd.to_datetime(df_corrigido_final['detected'])

# Lista de contadores
colunas_contadores = [col for col in df_corrigido_final.columns if col.startswith('count_')]

def calcular_sr_si_plotly(df, coluna='count_'):
    df = df.copy()
    df = df[['detected', coluna]].rename(columns={'detected': 'datetime', coluna: 'valor'})
    df['datetime'] = pd.to_datetime(df['datetime'])

    df['ano'] = df['datetime'].dt.isocalendar().year
    df['semana'] = df['datetime'].dt.isocalendar().week

    # SR por semana
    df_sr_semana = df.groupby('semana')['valor'].agg(['mean', 'max']).reset_index()
    df_sr_semana['SR'] = df_sr_semana['max'] / df_sr_semana['mean']

    # SR por ano/semana
    df_sr_ano_semana = df.groupby(['ano', 'semana'])['valor'].agg(['mean', 'max']).reset_index()
    df_sr_ano_semana['SR'] = df_sr_ano_semana['max'] / df_sr_ano_semana['mean']

    # SI por ano
    resultados_si = []
    for ano, grupo in df_sr_ano_semana.groupby('ano'):
        media_sr = grupo['SR'].mean()
        max_sr = grupo['SR'].max()
        si = media_sr / max_sr if max_sr != 0 else None
        resultados_si.append({'ano': ano, 'SI': si, 'SR_medio': media_sr, 'SR_max': max_sr})
    df_si = pd.DataFrame(resultados_si)

    return df_sr_ano_semana, df_sr_semana, df_si


In [24]:
# Dash app
app = Dash(__name__)

app.layout = html.Div([
    html.H2("Contagens Corrigidas e Indicadores de Sazonalidade"),

    dcc.Dropdown(
        id='dropdown-contador',
        options=[{'label': col, 'value': col} for col in colunas_contadores],
        value=colunas_contadores[0],
        clearable=False,
        style={'width': '50%'}
    ),

    dcc.Graph(id='grafico-contador'),

    html.H4("Seasonality Ratio (SR) por Semana e Ano"),
    dcc.Graph(id='grafico-sr'),

    html.H4("Seasonality Indicator (SI) por Ano"),
    dcc.Graph(id='grafico-si'),
])

@app.callback(
    Output('grafico-contador', 'figure'),
    Output('grafico-sr', 'figure'),
    Output('grafico-si', 'figure'),
    Input('dropdown-contador', 'value')
)
def atualizar_graficos(coluna_contador):
    # Gráfico de contagens
    fig_contador = px.line(
        df_corrigido_final,
        x='detected',
        y=coluna_contador,
        title=f"Contador Corrigido – {coluna_contador}",
        labels={'detected': 'Data', coluna_contador: 'Contagem'}
    )
    fig_contador.update_layout(height=350)

    # Cálculo de SR/SI
    df_sr_ano_semana, _, df_si = calcular_sr_si_plotly(df_corrigido_final, coluna=coluna_contador)

    # Gráfico SR por semana e ano
    fig_sr = px.line(
        df_sr_ano_semana,
        x='semana',
        y='SR',
        color='ano',
        title='SR por Semana e Ano',
        labels={'semana': 'Semana', 'SR': 'Seasonality Ratio', 'ano': 'Ano'}
    )
    fig_sr.update_layout(height=400)

    # Gráfico SI por ano
    fig_si = px.line(
        df_si,
        x='ano',
        y='SI',
        markers=True,
        title='SI por Ano',
        labels={'ano': 'Ano', 'SI': 'Seasonality Indicator'}
    )
    fig_si.add_hline(y=1, line_dash="dash", line_color="red")
    fig_si.update_layout(height=350)

    return fig_contador, fig_sr, fig_si

app.run(debug=True)
 # ou debug=True fora do notebook


In [19]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import pandas as pd

# Garantir que a coluna de data é datetime
df_corrigido_final['detected'] = pd.to_datetime(df_corrigido_final['detected'])

# Identificar colunas dos contadores
colunas_contadores = [col for col in df_corrigido_final.columns if col.startswith('count_')]

app = Dash(__name__)

app.layout = html.Div([
    html.H2("Contagens Corrigidas (MSTL) por Contador"),

    dcc.Dropdown(
        id='dropdown-contador',
        options=[{'label': col, 'value': col} for col in colunas_contadores],
        value=colunas_contadores[0],
        clearable=False,
        style={'width': '50%'}
    ),

    dcc.Graph(id='grafico-contador')
])

@app.callback(
    Output('grafico-contador', 'figure'),
    Input('dropdown-contador', 'value')
)
def atualizar_grafico(coluna_contador):
    fig = px.line(
        df_corrigido_final,
        x='detected',
        y=coluna_contador,
        title=f"Contador Corrigido – {coluna_contador}",
        labels={'detected': 'Data', coluna_contador: 'Contagem'}
    )
    fig.update_layout(height=400)
    return fig

app.run(mode='inline')  # para notebook. Se for em .py: use app.run_server(debug=True)



